# Prep

In [1]:
!pip install datasets diffusers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import os
from diffusers.schedulers import DDIMScheduler, DDPMScheduler, DPMSolverMultistepScheduler
from diffusers import StableDiffusionPipeline
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline, DDIMScheduler, DDPMScheduler, DPMSolverMultistepScheduler, UNet2DConditionModel
from diffusers.utils import make_image_grid
from string import whitespace
from google.colab import userdata, drive, files
from collections import OrderedDict
from safetensors.torch import load_file
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

import torch
import peft
import shutil
import tensorflow as tf
import datetime
import itertools
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import sys
from diffusers import AutoencoderKL, AutoPipelineForText2Image
import prep_utils
import inference_utils

In [9]:
# del sys.modules['inference_utils']

In [3]:
drive.mount('/content/drive') # access google drive

Mounted at /content/drive


In [4]:
os.environ['HF_TOKEN']= userdata.get('Huggingface')

In [13]:
models_parent_dir= '/content/drive/My Drive/results' # models dir setzen
os.environ['MODELS_PARENT_DIR'] = models_parent_dir

inference_dir = '/content/drive/My Drive/inference'
# inference_dir = "/content/test"
methods = ['textinv', 'dbooth', 'dlora', 'lora']
# subject_list = ['hannah', 'celine', 'timh', 'timk', 'leander', 'patrick', 'christoph', 'jannik', 'marco', 'nils']
# subject_list = ['timh', 'timk', 'leander', 'patrick', 'christoph', 'jannik', 'marco', 'nils']
subject_list = ['hannah', 'celine']

# subject_list = ['timh', 'celine']

models = ['SDv1.5', 'SDXL']
schedulers = [DDIMScheduler, DDPMScheduler, DPMSolverMultistepScheduler]

In [6]:
# Check if the folder exists
if not os.path.exists(inference_dir):
    # Create the folder
    os.makedirs(inference_dir)
    print(f"Folder created at {inference_dir}")
else:
    print(f"Folder already exists at {inference_dir}")

Folder already exists at /content/drive/My Drive/inference


# SD 1.5 Inference

In [ ]:
# test_dir = "/content/test"

In [ ]:
# !mkdir "/content/test"

mkdir: cannot create directory ‘/content/test’: File exists


In [ ]:
# shutil.rmtree("/content/test")

### Run Inference Grids & Decide on Hyperparameters

In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_lora_sd15(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=1200, #ggf. pick different checkpoints depending on subject?
                                           prompt=f"business headshot of a ohwx {prep_utils.get_gender(subject)}, business clothing, neutral background",
                                           scheduler=schedulers[1]
                                           )

In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_dlora_sd15(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=1500,
                                           prompt=f"business headshot of a ohwx {prep_utils.get_gender(subject)}",
                                           scheduler=schedulers[1]
                                           )

In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_dbooth_sd15(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=800, # 800 seems decent (still a bit overtrained with hannah, but 700 seems to weak? but celine's seems ok), 1200+ seems overtrained (cant make business clothing, seems to have a lot of the training image backgrounds)
                                           prompt=f"business headshot of a ohwx {prep_utils.get_gender(subject)}",
                                           scheduler=schedulers[1]
                                           )


In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_textinv_sd15(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=1500, #Checkpoint is required for this method
                                           prompt=f"business headshot of a <sks> {prep_utils.get_gender(subject)}, business clothing, neutral background",
                                           scheduler=schedulers[1]
                                           )

### Run SD1.5 Inference for all methods

In [7]:
for subject in subject_list:
  inference_utils.run_lora_sd15_inference(subject,
                          save_dir=inference_dir,
                          checkpoint=1200,
                          num_inference_steps=40,
                          guidance_scale= 7,
                          scheduler=schedulers[1])

  # inference_utils.run_dlora_sd15_inference(subject,
  #                         save_dir=inference_dir,
  #                         checkpoint=1500,
  #                         num_inference_steps=30,
  #                         guidance_scale= 6,
  #                         scheduler=schedulers[1])

  # inference_utils.run_dbooth_sd15_inference(subject,
  #                         save_dir=inference_dir,
  #                         checkpoint=1200,
  #                         num_inference_steps=40,
  #                         guidance_scale= 7,
  #                         scheduler=schedulers[1])

  # inference_utils.run_textinv_sd15_inference(subject,
  #                         save_dir=inference_dir,
  #                         checkpoint=1500, # Checkpoint is required for this method
  #                         num_inference_steps=40,
  #                         guidance_scale= 7,
  #                         scheduler=schedulers[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/40 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/40 [00:00<?, ?it/s]

# SDXL Inference

## Run Inference Grids & Decide on Hyperparameters

In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_lora_sdxl(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=800, # 1000 seems to have overfit,
                                           prompt=f"business headshot of a ohwx {prep_utils.get_gender(subject)}, business clothing, neutral background",
                                           scheduler=schedulers[1]
                                           )

In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_dlora_sdxl(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=None, #latest checkpoint seems ok?
                                           prompt=f"business headshot of a ohwx {prep_utils.get_gender(subject)}, business clothing, neutral background",
                                           scheduler=schedulers[1]
                                           )

In [ ]:
for subject in subject_list:
  images, plots= inference_utils.generate_inference_grid_textinv_sdxl(subject=subject,
                                           save_dir=inference_dir,
                                           checkpoint=None,
                                           prompt=f"business headshot of a <sks> {prep_utils.get_gender(subject)}, business clothing, neutral background",
                                           scheduler=schedulers[1]
                                           )

Output hidden; open in https://colab.research.google.com to view.

## Run SDXL Inference for all methods

In [8]:
for subject in subject_list:
  # inference_utils.run_lora_sdxl_inference(subject,
  #                         save_dir=inference_dir,
  #                         checkpoint=800,
  #                         num_inference_steps=40,
  #                         guidance_scale=6,
  #                         scheduler=schedulers[1]
  #                         )
  # inference_utils.run_dlora_sdxl_inference(subject,
  #                         save_dir=inference_dir,
  #                         checkpoint=None,
  #                         num_inference_steps=40,
  #                         guidance_scale=6,
  #                         scheduler=schedulers[1]
  #                         )

  inference_utils.run_textinv_sdxl_inference(subject,
                          save_dir=inference_dir,
                          checkpoint=None,
                          num_inference_steps=40,
                          guidance_scale=7,
                          scheduler=schedulers[1]
                          )

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

# Flux Inference

Guidance-distilled inference variant of flux implemented here does not necessitate inference grid, as 50 sampling steps generally provide good quality results. Nevertheless, it offers additional insight.

## Run Inference Grid & Decide on Hyperparameters

In [7]:
for subject in subject_list:
  inference_utils.generate_inference_grid_lora_flux(subject,
                                    save_dir=inference_dir,
                                    checkpoint=None,
                                    prompt=f"business headshot of a ohwx {prep_utils.get_gender(subject)}, business clothing, neutral background"
                                    )

Output hidden; open in https://colab.research.google.com to view.

## Run Flux Inference

In [14]:
for subject in subject_list:
  inference_utils.run_lora_flux_inference(subject,
                          save_dir=inference_dir,
                          checkpoint=None,
                          num_inference_steps=50, # =50 required due to preset scheduler
                          guidance_scale=5
  )

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]